In [6]:
import pandas as pd
import os
import glob
import shutil
import random
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
train_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-train-canonical/'
# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
dev_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-dev-canonical/'

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-candidates-dwd-v2/
train_candidate_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-train-candidates-dwd-v2/'
# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-candidates-dwd-v2/
dev_candidate_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-dev-candidates-dwd-v2/'

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
ground_truth_files = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/Xinting_GT_csv/round_1/'

In [3]:
aux_field = 'graph_embedding_complex,class_count,property_count'
temp_dir = './temp' #temp directory to store intermediate files

#directory to store the property count file for each table. Can be directly used for computing the tf-idf features 
#without running the candidate generation process again which is expensive

#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/train_prop_count/
train_prop_count = './train_prop_count/' 
#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/dev_prop_count/
dev_prop_count = './dev_prop_count/'

#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/train_class_count/
train_class_count = './train_class_count/'
#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/dev_class_count/
dev_class_count = './dev_class_count/'


!mkdir -p $temp_dir
!mkdir -p $train_prop_count
!mkdir -p $dev_prop_count
!mkdir -p $train_class_count
!mkdir -p $dev_class_count
candidates = os.path.join(temp_dir,'candidates.csv')
embedding_file = os.path.join(temp_dir, 'graph_embedding_complex.tsv')
print(candidates)

./temp/candidates.csv


In [40]:
def cand_feat_generation(path, gt_path, output_path, class_count, prop_count):
    for file in glob.glob(path + '*.csv')[-1:]:
        st = time.time()
        filename = file.split('/')[-1]
        print(filename)
        gt_file = os.path.join(ground_truth_files, filename)
        output_file = os.path.join(output_path, filename)
        
        !tl clean -c label -o label_clean $file / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder $temp_dir / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder {temp_dir} / \
        ground-truth-labeler --gt-file $gt_file > $candidates
        
        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = os.path.join(class_count, filename.strip('.csv') + '_class_count.tsv')
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = os.path.join(prop_count, filename.strip('.csv') + '_prop_count.tsv')
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            else:
                aux_df.to_csv(f'{temp_dir}/{field}.tsv', sep='\t', index=False)
        
        !tl string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan $candidates \
            / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
            / string-similarity -i --method jaro_winkler -o jaro_winkler \
            / score-using-embedding --column-vector-strategy centroid-of-singletons -o graph-embedding-score \
            --embedding-file $embedding_file \
            / create-singleton-feature -o singleton\
            / generate-reciprocal-rank -c graph-embedding-score -o reciprocal_rank\
            / mosaic-features -c kg_labels --num-char --num-tokens > $output_file
        
        print(time.time() - st)



In [ ]:
cand_feat_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count)

In [41]:
candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count)

14067031_0_559833072073397908.csv
Qnodes to lookup: 7693
Qnodes from file: 7438
412.62110209465027


### Generate Balanced Training Data

In [49]:
training_datapath = '../random_forest_ranking/training_data_dwd.csv'

In [43]:
final_list = []
for i,file in enumerate(glob.glob(train_candidate_path + '*.csv')):
    file_name = file.split('/')[-1]
    print(file_name)
    
    try:
        d_sample = pd.read_csv(file)
        grouped_obj = d_sample.groupby(['row', 'column'])
        for cell in grouped_obj:
            num_rows = random.randint(2,5)
            sorted_df = cell[1].sort_values('graph-embedding-score',ascending=False)
            if 0 in sorted_df['evaluation_label'].tolist():
                continue
            if sorted_df.empty:
                continue
            if num_rows < len(sorted_df):
                top_sample_df = sorted_df[sorted_df['evaluation_label'] == -1][:10].sample(n=num_rows)
                bottom_sample_df = sorted_df[sorted_df['evaluation_label'] == -1][-10:].sample(n=num_rows)
                final_list.extend(top_sample_df.to_dict(orient='records'))
                final_list.extend(bottom_sample_df.to_dict(orient='records'))
            else:
                sample_df = sorted_df[sorted_df['evaluation_label'] == -1]
                final_list.extend(sample_df.to_dict(orient='records'))
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            final_list.extend(a.to_dict(orient='records'))
    except: 
        pass

train_df = pd.DataFrame(final_list)

58891288_0_1117541047012405958.csv
39173938_0_7916056990138658530.csv
10579449_0_1681126353774891032.csv
33401079_0_9127583903019856402.csv
21362676_0_6854186738074119688.csv
38428277_0_1311643810102462607.csv
91959037_0_7907661684242014480.csv
20135078_0_7570343137119682530.csv
35188621_0_6058553107571275232.csv
54719588_0_8417197176086756912.csv
21245481_0_8730460088443117515.csv
71840765_0_6664391841933033844.csv
8468806_0_4382447409703007384.csv
88523363_0_8180214313099580515.csv
29414811_13_8724394428539174350.csv
99070098_0_2074872741302696997.csv
43237185_1_3636357855502246981.csv
46671561_0_6122315295162029872.csv
53989675_0_8697482470743954630.csv
25404227_0_2240631045609013057.csv
9834884_0_3871985887467090123.csv
63450419_0_8012592961815711786.csv
1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5.csv
22864497_0_8632623712684511496.csv
53822652_0_5767892317858575530.csv
37856682_0_6818907050314633217.csv
26310680_0_5150772059999313798.csv
29414811_12_2511524702

In [50]:
train_df.to_csv(training_datapath, index=False)

### Data Exploration

In [16]:
train_datapath = '../random_forest_ranking/training_data_dwd.csv'

In [17]:
df = pd.read_csv(train_datapath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   column                 48365 non-null  int64  
 1   row                    48365 non-null  int64  
 2   label                  48363 non-null  object 
 3   context                48353 non-null  object 
 4   label_clean            48363 non-null  object 
 5   kg_id                  47784 non-null  object 
 6   kg_labels              46828 non-null  object 
 7   kg_aliases             14414 non-null  object 
 8   method                 48365 non-null  object 
 9   kg_descriptions        39166 non-null  object 
 10  pagerank               48365 non-null  float64
 11  retrieval_score        48365 non-null  float64
 12  GT_kg_id               48365 non-null  object 
 13  GT_kg_label            48365 non-null  object 
 14  evaluation_label       48365 non-null  int64  
 15  mo

In [18]:
# Features we need to include in training
features = ['pagerank','retrieval_score','monge_elkan',
            'des_cont_jaccard','jaro_winkler','graph-embedding-score',
           'singleton','num_char','num_tokens','reciprocal_rank']
evaluation_label = ['evaluation_label']

df[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pagerank               48365 non-null  float64
 1   retrieval_score        48365 non-null  float64
 2   monge_elkan            48365 non-null  float64
 3   des_cont_jaccard       48365 non-null  float64
 4   jaro_winkler           48365 non-null  float64
 5   graph-embedding-score  48365 non-null  float64
 6   singleton              48365 non-null  int64  
 7   num_char               48365 non-null  int64  
 8   num_tokens             48365 non-null  int64  
 9   reciprocal_rank        48365 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 3.7 MB


In [57]:
df['graph-embedding-score'] = df['graph-embedding-score'].fillna(0.0)
df['reciprocal_rank'] = df['reciprocal_rank'].fillna(0.0)
df[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pagerank               48365 non-null  float64
 1   retrieval_score        48365 non-null  float64
 2   monge_elkan            48365 non-null  float64
 3   des_cont_jaccard       48365 non-null  float64
 4   jaro_winkler           48365 non-null  float64
 5   graph-embedding-score  48365 non-null  float64
 6   singleton              48365 non-null  int64  
 7   num_char               48365 non-null  int64  
 8   num_tokens             48365 non-null  int64  
 9   reciprocal_rank        48365 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 3.7 MB


### Train a Random Forest Regressor

In [58]:
train_data = df[features]
y_label = df[evaluation_label]

In [61]:
model = RandomForestRegressor(n_estimators=100, max_features="log2",min_samples_leaf=3)
model.fit(train_data,y_label)
y_pred = model.predict(train_data)

/Users/rijulvohra/opt/anaconda3/envs/table_linker_dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [62]:
mean_squared_error(y_label, y_pred)

0.036428866339495325

In [8]:
model_save_path = '../random_forest_ranking/rf_tuned_dwd_ranking.pkl'


In [ ]:
pickle.dump(model,open(model_save_path,'wb'))

In [9]:
saved_model = pickle.load(open(model_save_path, 'rb'))

### Predicting Scores for Train set

In [71]:
train_candidate_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-train-candidates-dwd-v2/'
train_pred_output = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-train-rf-pred-dwd-2/'

In [75]:
train_mse = []
for file in glob.glob(train_candidate_path + '*.csv'):
    try:
        file_name = file.split('/')[-1]
        print(file_name)
        df_file = pd.read_csv(file)
        data = df_file[features]
        y_file_label = df_file[evaluation_label]
        y_file_pred = saved_model.predict(data)
        df_file['rf_model_pred'] = y_file_pred
        file_mse = mean_squared_error(y_file_label,y_file_pred)
        train_mse.append(file_mse)
        df_file.to_csv(os.path.join(train_pred_output,file_name),index=False)
    except:
        pass

58891288_0_1117541047012405958.csv
39173938_0_7916056990138658530.csv
10579449_0_1681126353774891032.csv
33401079_0_9127583903019856402.csv
21362676_0_6854186738074119688.csv
38428277_0_1311643810102462607.csv
91959037_0_7907661684242014480.csv
20135078_0_7570343137119682530.csv
35188621_0_6058553107571275232.csv
54719588_0_8417197176086756912.csv
21245481_0_8730460088443117515.csv
71840765_0_6664391841933033844.csv
8468806_0_4382447409703007384.csv
88523363_0_8180214313099580515.csv
29414811_13_8724394428539174350.csv
99070098_0_2074872741302696997.csv
43237185_1_3636357855502246981.csv
46671561_0_6122315295162029872.csv
53989675_0_8697482470743954630.csv
25404227_0_2240631045609013057.csv
9834884_0_3871985887467090123.csv
63450419_0_8012592961815711786.csv
1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5.csv
22864497_0_8632623712684511496.csv
53822652_0_5767892317858575530.csv
37856682_0_6818907050314633217.csv
26310680_0_5150772059999313798.csv
29414811_12_2511524702

In [76]:
print("Train MSE is: ", sum(train_mse)/len(train_mse))

Train MSE is:  0.4519201624197591


### Predicting Scores for dev set

In [10]:
dev_candidate_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-dev-candidates-dwd-v2/'
dev_pred_output = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-dev-rf-pred-dwd-2/'

In [19]:
dev_mse = []
for file in glob.glob(dev_candidate_path + '*.csv'):

    file_name = file.split('/')[-1]
    print(file_name)
    df_file = pd.read_csv(file)
    data = df_file[features]
    y_file_label = df_file[evaluation_label]
    y_file_pred = saved_model.predict(data)
    df_file['rf_model_pred'] = y_file_pred
    file_mse = mean_squared_error(y_file_label,y_file_pred)
    dev_mse.append(file_mse)
    df_file.to_csv(os.path.join(dev_pred_output,file_name),index=False)
    

39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


In [20]:
print("Dev MSE is: ", sum(dev_mse)/len(dev_mse))

Dev MSE is:  0.40129949369858403


### Evaluation

In [21]:
final_score_path = dev_pred_output

In [22]:
import os
eval_file_names = []
for (dirpath, dirnames, filenames) in os.walk(final_score_path):
    for fn in filenames:
        if "csv" not in fn:
            continue
        abs_fn = dirpath + fn
        assert os.path.isfile(abs_fn)
        if os.path.getsize(abs_fn) == 0:
            continue
        eval_file_names.append(abs_fn)
len(eval_file_names)

8

In [23]:
# merge all eval files in one df
def merge_df(file_names: list):
    df_list = []
    for fn in file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        # df = df.fillna('')
        df_list.append(df)
    return pd.concat(df_list)
all_data = merge_df(eval_file_names)
all_data

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,...,monge_elkan,des_cont_jaccard,jaro_winkler,graph-embedding-score,singleton,reciprocal_rank,num_char,num_tokens,rf_model_pred,table_id
0,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q185888,The Social Network,Social Network,fuzzy-augmented,2010 film by David Fincher,...,1.000000,0.400000,1.000000,0.771462,0,1.000000,18,3,0.993595,39759273_0_1427898308030295194
1,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q1952928,The Social Network,NaN,fuzzy-augmented,2010 soundtrack album by Trent Reznor and Atti...,...,1.000000,0.111111,1.000000,0.522352,0,0.083333,18,3,-0.040517,39759273_0_1427898308030295194
2,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q2715623,social network,social networking,fuzzy-augmented,theoretical concept in sociology,...,0.915344,0.000000,0.830688,0.142025,0,0.011765,14,2,-0.496516,39759273_0_1427898308030295194
3,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q86362445,Libbie Higgins,Carla Higgins|The White Trash Network,fuzzy-augmented,American comedian,...,0.390212,0.000000,0.478307,0.394119,0,0.018519,14,2,0.022884,39759273_0_1427898308030295194
4,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q4769433,Annie Social,NaN,fuzzy-augmented,American professional wrestler and professiona...,...,0.605159,0.000000,0.578704,0.277974,0,0.013699,12,2,-0.278810,39759273_0_1427898308030295194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,0,9,Chad,"Arabic, French|41.3|Aug 1960|Bichara Cherif Da...",Chad,Q629768,Yasutora Sado,Chad|Sado Yasutora,exact-match,fictional character from Bleach,...,0.614583,0.000000,0.442308,-0.033865,0,0.005650,13,2,-1.000000,14067031_0_559833072073397908
8134,0,9,Chad,"Arabic, French|41.3|Aug 1960|Bichara Cherif Da...",Chad,Q63882300,Chad,NaN,exact-match,musical character in the musical All Shook Up ...,...,1.000000,0.000000,1.000000,0.119076,0,0.006536,4,1,-0.948734,14067031_0_559833072073397908
8135,0,9,Chad,"Arabic, French|41.3|Aug 1960|Bichara Cherif Da...",Chad,Q63892906,Chad,NaN,exact-match,musical character in the musical The Kid,...,1.000000,0.000000,1.000000,0.119267,0,0.006623,4,1,-0.972305,14067031_0_559833072073397908
8136,0,9,Chad,"Arabic, French|41.3|Aug 1960|Bichara Cherif Da...",Chad,Q657,Chad,🇹🇩|GUM|Republic of Chad|CHA|td,exact-match,sovereign state in central Africa,...,1.000000,0.000000,1.000000,0.676664,0,0.500000,4,1,0.999298,14067031_0_559833072073397908


In [24]:
# parse eval file
from pandas.core.common import SettingWithCopyError
pd.options.mode.chained_assignment = 'raise'

def parse_eval_files_stats(eval_data,method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'row', 'column'])['table_id'].count().reset_index(name="count")
    res['num_tasks'] = len(eval_data.groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt_in_candidate'] = len(eval_data[eval_data['evaluation_label'] == 1].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_singleton_candidate'] = len(candidate_eval_data[candidate_eval_data['count'] == 1].groupby(['table_id', 'row', 'column']))
    singleton_eval_data = candidate_eval_data[candidate_eval_data['count'] == 1]
    num_tasks_with_singleton_candidate_with_gt = 0
    for i, row in singleton_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) == 1
        if c_e_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_singleton_candidate_with_gt += 1
    res['num_tasks_with_singleton_candidate_with_gt'] = num_tasks_with_singleton_candidate_with_gt
    num_tasks_with_graph_top_one_accurate = []
    num_tasks_with_graph_top_five_accurate = []
    num_tasks_with_graph_top_ten_accurate = []
    num_tasks_with_final_score_top_one_accurate = []
    num_tasks_with_final_score_top_five_accurate = []
    num_tasks_with_final_score_top_ten_accurate = []
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    ndcg_score_g_list = []
    ndcg_model_score_list = []
    has_gt_list = []
    has_gt_in_candidate = []
    # candidate_eval_data = candidate_eval_data[:1]
    for i, row in candidate_eval_data.iterrows():
        #print(i)
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
            
        # handle graph-embedding-score
        s_data = c_e_data.sort_values(by=['graph-embedding-score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_graph_top_one_accurate.append(1)
        else:
            num_tasks_with_graph_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_graph_top_five_accurate.append(1)
        else:
            num_tasks_with_graph_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_graph_top_ten_accurate.append(1)
        else:
            num_tasks_with_graph_top_ten_accurate.append(0)
        
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
        cf_e_data = c_e_data.copy()
        #cf_e_data['evaluation_label'] = cf_e_data['evaluation_label'].replace(-1, 0)
#         cf_e_data['text-embedding-score'] = cf_e_data['text-embedding-score'].replace(np.nan, 0)
        cf_e_data['graph-embedding-score'] = cf_e_data['graph-embedding-score'].replace(np.nan, 0)
        cf_e_data[method] = cf_e_data[method].replace(np.nan, 0)

    candidate_eval_data['graph_top_one_accurate'] = num_tasks_with_graph_top_one_accurate
    candidate_eval_data['graph_top_five_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['graph_top_ten_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['model_top_one_accurate'] = num_tasks_with_model_score_top_one_accurate
    candidate_eval_data['model_top_five_accurate'] = num_tasks_with_model_score_top_five_accurate
    candidate_eval_data['model_top_ten_accurate'] = num_tasks_with_model_score_top_ten_accurate
    candidate_eval_data['has_gt'] = has_gt_list
    candidate_eval_data['has_gt_in_candidate'] = has_gt_in_candidate
    res['num_tasks_with_graph_top_one_accurate'] = sum(num_tasks_with_graph_top_one_accurate)
    res['num_tasks_with_graph_top_five_accurate'] = sum(num_tasks_with_graph_top_five_accurate)
    res['num_tasks_with_graph_top_ten_accurate'] = sum(num_tasks_with_graph_top_ten_accurate)
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [25]:
res, candidate_eval_data = parse_eval_files_stats(all_data,'rf_model_pred')
print(res)
display(candidate_eval_data)

{'num_tasks': 714, 'num_tasks_with_gt': 708, 'num_tasks_with_gt_in_candidate': 695, 'num_tasks_with_singleton_candidate': 0, 'num_tasks_with_singleton_candidate_with_gt': 0, 'num_tasks_with_graph_top_one_accurate': 289, 'num_tasks_with_graph_top_five_accurate': 491, 'num_tasks_with_graph_top_ten_accurate': 552, 'num_tasks_with_model_score_top_one_accurate': 513, 'num_tasks_with_model_score_top_five_accurate': 595, 'num_tasks_with_model_score_top_ten_accurate': 616}


,table_id,row,column,count,graph_top_one_accurate,graph_top_five_accurate,graph_top_ten_accurate,model_top_one_accurate,model_top_five_accurate,model_top_ten_accurate,has_gt,has_gt_in_candidate
0,14067031_0_559833072073397908,0,0,172,1,1,1,1,1,1,1,1
1,14067031_0_559833072073397908,1,0,169,1,1,1,1,1,1,1,1
2,14067031_0_559833072073397908,2,0,158,1,1,1,1,1,1,1,1
3,14067031_0_559833072073397908,3,0,144,0,0,0,0,0,0,1,0
4,14067031_0_559833072073397908,4,0,157,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
709,84575189_0_6365692015941409487,95,2,100,0,0,0,1,1,1,1,1
710,84575189_0_6365692015941409487,96,2,101,0,0,0,1,1,1,1,1
711,84575189_0_6365692015941409487,97,2,101,0,0,0,1,1,1,1,1
712,84575189_0_6365692015941409487,98,2,104,0,0,0,0,0,0,1,1


In [26]:
# Conclusion of exact-match on all tasks with ground truth (no filtering)
print(f"number of tasks: {res['num_tasks']}")
print(f"number of tasks with ground truth: {res['num_tasks_with_gt']}")
print(f"number of tasks with ground truth in candidate set: {res['num_tasks_with_gt_in_candidate']}, which is {res['num_tasks_with_gt_in_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set: {res['num_tasks_with_singleton_candidate']}, which is {res['num_tasks_with_singleton_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set which is ground truth: {res['num_tasks_with_singleton_candidate_with_gt']}, which is {res['num_tasks_with_singleton_candidate_with_gt']/res['num_tasks_with_gt'] * 100}%")
print()
print(f"number of tasks with top-1 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_one_accurate']}, which is {res['num_tasks_with_graph_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_five_accurate']}, which is {res['num_tasks_with_graph_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_ten_accurate']}, which is {res['num_tasks_with_graph_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
print(f"number of tasks with top-1 accuracy in terms of model score: {res['num_tasks_with_model_score_top_one_accurate']}, which is {res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of model score: {res['num_tasks_with_model_score_top_five_accurate']}, which is {res['num_tasks_with_model_score_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of model score: {res['num_tasks_with_model_score_top_ten_accurate']}, which is {res['num_tasks_with_model_score_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
candidate_eval_data_with_gt = candidate_eval_data[candidate_eval_data['has_gt'] == 1]

number of tasks: 714
number of tasks with ground truth: 708
number of tasks with ground truth in candidate set: 695, which is 98.1638418079096%
number of tasks has singleton candidate set: 0, which is 0.0%
number of tasks has singleton candidate set which is ground truth: 0, which is 0.0%

number of tasks with top-1 accuracy in terms of graph embedding score: 289, which is 40.81920903954802%
number of tasks with top-5 accuracy in terms of graph embedding score: 491, which is 69.35028248587571%
number of tasks with top-10 accuracy in terms of graph embedding score: 552, which is 77.96610169491525%

number of tasks with top-1 accuracy in terms of model score: 513, which is 72.45762711864407%
number of tasks with top-5 accuracy in terms of model score: 595, which is 84.03954802259888%
number of tasks with top-10 accuracy in terms of model score: 616, which is 87.00564971751412%



In [27]:
c = candidate_eval_data.groupby(['table_id']).agg({
    'graph_top_one_accurate':lambda x: sum(x)/len(x),
    'model_top_one_accurate':lambda x: sum(x)/len(x),
    'graph_top_five_accurate':lambda x: sum(x)/len(x),
    'model_top_five_accurate':lambda x: sum(x)/len(x)
})
c['table type'] = [
    'country II',
    'companies',
    'pope',
    'video games',
    'movies',
    'players I',
    'players II',
    'magazines'
]
c

,graph_top_one_accurate,model_top_one_accurate,graph_top_five_accurate,model_top_five_accurate,table type
table_id,,,,,
14067031_0_559833072073397908,0.886792,0.943396,0.943396,0.943396,country II
14380604_4_3329235705746762392,0.700000,0.700000,0.900000,0.800000,companies
28086084_0_3127660530989916727,0.236607,0.611607,0.522321,0.723214,pope
29414811_2_4773219892816395776,0.136364,0.545455,0.500000,0.863636,video games
39759273_0_1427898308030295194,0.580000,0.880000,0.930000,0.970000,movies
45073662_0_3179937335063201739,0.407407,0.629630,0.888889,0.740741,players I
50270082_0_444360818941411589,0.547619,0.744048,0.922619,0.910714,players II
84575189_0_6365692015941409487,0.110000,0.700000,0.230000,0.780000,magazines
